<h2>Charts and Analysis of Survey Data</h2>

Starts with input file form comment prep.


In [3]:
import os
import sys
import matplotlib
import pandas as pd

#filename = os.path.join('..', 'src_data', 'town services and housing costs public.xlsx')
#df = pd.read_excel(filename, encoding='utf-8', convert_float=True)

# TBD not as checked in:
filename = os.path.join('..', 'src_data', 'town services and housing costs public.csv')
df2 = pd.read_csv(filename, encoding='utf-8')
print(df2.info())
df=df2

print(df.info())


FileNotFoundError: [Errno 2] File b'..\\src_data\\town services and housing costs public.csv' does not exist: b'..\\src_data\\town services and housing costs public.csv'

In [ ]:
#
# starting point for all plots
#
plotdf = df[df.ResponsibleCode == 1].copy()
print(plotdf.info())
# precaution
#df = None
print ('Ready. Done')

<h3>Plot Common Code</h3>
Run this one section before running any plot sections
    

In [ ]:
#
# annotation example from: https://stackoverflow.com/questions/23591254/python-pandas-matplotlib-annotating-labels-above-bar-chart-columns
#
import matplotlib.pyplot as plt
import numpy as np
import os

chart_dir = os.path.join('lexcharts', 'relationships')
freq_chart_dir = os.path.join('lexcharts', 'frequencies')

def annotate_patches_float(stacked, patches, precision=1, label_fontsize=22, dual_annotation=False):
    annotate_string = '%2.' + str(precision) + 'f%%'
    # print'calling annotate patches'
    return annotate_patches(stacked, patches, annotate_string, multiplier=100.0,
                           label_fontsize=label_fontsize, dual_annotation=dual_annotation)
#    return annotate_patches(stacked, patches, '%2.1f%%', 100.0)

def annotate_patches_integer(stacked, patches, dual_annotation=False):
    return annotate_patches(stacked, patches, '%d', 1.0, 22, dual_annotation)

def annotate_patches(stacked, patches, formatter, multiplier, label_fontsize=22, dual_annotation=False):
    """
    general routing to annotate patches
    
    patches - pass in ax.patches
    formatter: '%2.1f%%' for percent and '%d' for integer
    """
    global ax
    if stacked:
        print('is stacked')
        if dual_annotation:
            assume_height_placement_offset = 45
        else:
            assume_height_placement_offset = 15
        if len(patches)>10:
            # adjust for charts with many patches
            assume_height_placement_offset = assume_height_placement_offset / 2
        label_horiz_adj = label_fontsize * 2.0 / 3.0
        cum_value = 0
        for p in patches:
            small_bar_adjustment = 0
            if p.get_width() < 50:
                small_bar_adjustment = p.get_width()+12  # shift over the whole bar length so figure appears to right of bar
            cum_value += p.get_width()
            text_str = formatter % (p.get_width()*multiplier)
            text_len_offset = (1.0*len(text_str))/2.0
            if p.get_width() > 0:  # only patches with width
                ax.annotate(text_str, (p.get_x() + p.get_width()/2.0, p.get_y()),
                            xytext=(-label_horiz_adj*text_len_offset + small_bar_adjustment, assume_height_placement_offset), 
                            fontsize=label_fontsize, textcoords='offset points')
        if dual_annotation:
            # show percentages as well
            assume_height_placement_offset = 15
            if len(patches)>10:
                # adjust for charts with many patches
                assume_height_placement_offset = assume_height_placement_offset / 2
            for p in patches:
                small_bar_adjustment = 0
                if p.get_width() < 50:
                    small_bar_adjustment = p.get_width()+12
                text_str = '%2.f%%' % (p.get_width()*100.0 / cum_value)
                text_len_offset = (1.0*len(text_str))/2.0
                if p.get_width() > 0:  # only patches with width
                    ax.annotate(text_str, (p.get_x() + p.get_width()/2.0, p.get_y()),
                                xytext=(-label_horiz_adj*text_len_offset + small_bar_adjustment, 
                                       assume_height_placement_offset), 
                                fontsize=label_fontsize-4, textcoords='offset points')
    else:
        if dual_annotation:
            y_offset = -3
        else:
            y_offset = 3
        cum_value = 0
        for p in patches:
            cum_value += p.get_width()
            text_str = formatter % (p.get_width()*multiplier)
            ax.annotate(text_str, (p.get_x() + p.get_width(), p.get_y()),
                        xytext=(0, y_offset), 
                        fontsize=label_fontsize, textcoords='offset points')
            #print 'x is %1.1f, y is %1.1f, %s' % ((p.get_x() + p.get_width()), p.get_y(), text_str)
        if dual_annotation:
            # show percentages as well
            for p in patches:
                text_str = '%2.f%%' % (p.get_width()*100.0 / cum_value)
                ax.annotate(text_str, (p.get_x() + p.get_width(), p.get_y()),
                            xytext=(0, -y_offset), 
                            fontsize=label_fontsize-4, textcoords='offset points')
            
        
def ylabel_range_annotation(axes, dollar_prefix=True):
    """
    convert ylabels into a range based on range of consecutive values
    
    dollar_prefix: whether to include a dollar prefix or not
    """
    new_labels = []
    old_label = ''
    for label in axes.get_yticklabels():
        if old_label != '':
            new_str = '{:,d}'.format(int(label.get_text())-1) 
            lead_str = ''
            if dollar_prefix:
                lead_str = '\$'
            new_labels.append('%s%s-%s' % (lead_str, old_label, new_str))
        old_label = '{:,d}'.format(int(label.get_text())) 
    new_labels.append('%s%s+' % (lead_str, old_label))
    axes.set_yticklabels(new_labels)

def save_figure(figure_name, override_chart_dir=''):
    """
    save figure in chart directory
    """
    global plt, chart_dir
    
    if override_chart_dir == '':
        override_chart_dir = chart_dir
    fig = plt.gcf()
    plt.tight_layout()
    filename = os.path.join(override_chart_dir, figure_name)
    print('Saving figure %s' % filename)
    fig.savefig(filename)
    plt.show()    
    
    
def sort_df_inplace(df, seq):
    '''
    use sequence to sort items
    create temp column then drop after sorting
    '''
    df['temp']=-1
    for cntr, item in enumerate(seq):
        df.loc[df.index==item,'temp']=cntr
    print('Assigned temps:')
    print (df.temp)
    if df[df.temp==-1].temp.count()>0:
        raise 'Not all values transformed'
    df.sort_values('temp', inplace=True, ascending=False)
    df.drop(columns = ['temp'], inplace=True)
        
    
def special_sort(df, field_name):
    '''
    custom sort fields if generic sort does not work
    '''
    if field_name == 'MonthlyHousingCost':
        seq = ['$10,001+', '$8,001-$10,000', '$6,001-$8,000', '$4,001-$6,000','$2,001-$4,000', '<$2,000/month', 'ResponseError']
        sort_df_inplace(df, seq)
    elif field_name == 'Stress':
        seq = ['Substantial', 'Significant', 'Some', 'Little', 'None']
        sort_df_inplace(df, seq)
    elif field_name == 'PropertyTaxShare':
        seq = ['Substantial (76-100%)','Significant (51-75%)', 'Some (26-50%)', 'Little (1-25%)', 'None']
        sort_df_inplace(df, seq)
    elif field_name == 'Relocation':
        seq = ['Definitely', 'Likely', 'Undecided - Possible', 'Have not considered', 'Unlikely', 'Definitely Not']
        sort_df_inplace(df, seq)
    elif field_name == 'RelocationBand':
        seq = ['Likely', 'Neutral', 'Unlikely']
        sort_df_inplace(df, seq)
    elif field_name == 'Tenure':
        seq = ['moved in 2015 or later', 'moved in 2010-2014', 'moved in 2000-2009', 'moved in 1990-1999', 'moved in 1980-1989', 'moved in 1979 and earlier']
        sort_df_inplace(df, seq)
    elif field_name == 'Age':
        seq = ['80 years and over', '70-79 years', '60-69 years', '50-59 years', '40-49 years', '31-39 years', 'under 30 years']
        sort_df_inplace(df, seq)
    elif field_name == 'HouseholdSize':
        seq = ['5+', '4', '3', '2', '1']
        sort_df_inplace(df, seq)
    elif field_name == 'PropertyType':
        seq = ['single family home', 'condominium', 'multi-family home', 'apartment', 'other']
        sort_df_inplace(df, seq)
    elif field_name == 'Income':
        seq = ['$200,000 and over', '$150,000-$199,999', '$100,000-$149,999', '$75,000-$99,000',
              '$50,000-$74,999', '$35,000-$49,999', 'Less than $35,000']
        sort_df_inplace(df, seq)
    elif field_name == 'MarketValue':
        # repsonse too large for reasonable printing
        df.rename(index = {"I don't live in a property owned by a family member" : 'Not owned by family'},
                  inplace=True)
        seq = ['$2,000,000 and more', '$1,750,000-$1,999,999', '$1,500,000-$1,749,999',
              '$1,250,000-$1,499,999', '$1,000,000-$1,249,999', '$750,000-$999,999',
              '$500,000-$749,999', 'Less than $500,000', 'Not owned by family']
        sort_df_inplace(df, seq)
    elif field_name == 'OwnRent':
        # do not want the -1 included
        df.drop('-1', inplace=True)
        seq = ['Own', 'Rent']
        sort_df_inplace(df, seq)
        
        
    else:
        print('default sorting')
        df.sort_index(inplace=True)
    
    
# standard frequency distribution
def freq_dist_plot(field_name, title=''):
    global plotdf, ax
    if title == '':
        title = field_name
    print(field_name)
    if 1==0:
        # debugging
        print(f'freq_dist_plot field: {field_name}')
        print(f'type of field: {type(plotdf[field_name])}')
        print ('Plot values >= 0')
        print(plotdf[(plotdf[field_name] >=0)][field_name])
        print ('Plot value counts')
        print(plotdf[(plotdf[field_name] >=0)][field_name].value_counts())
# old form    dfc = plotdf[(plotdf[field_name] >=0)][field_name].value_counts().to_frame('count')
    dfc = plotdf[plotdf[field_name].isnull()==False][field_name].value_counts().to_frame('count')
    special_sort(dfc, field_name)
    if dfc.shape[0] > 3:
        # ensure bars thick enough to put two levels of annotation text
        bar_width = 0.8
    else:
        # do not make overly fat bars
        bar_width = 0.5
    print(dfc.index.tolist())
    ax = dfc.plot.barh(stacked=False, fontsize=20, color='orange', width=bar_width, figsize=(12,12))
    adjusted_text = [item.get_text().replace('$', '\$') for item in ax.get_yticklabels()]
    ax.set_yticklabels(adjusted_text)
    plot_title('%s' % title, fontsize=24)
    ax.set_ylabel('%s' % title, fontsize=22)
    ax.set_xlabel('Response Count', fontsize=22)

    # pretend to be stacked so values can be in bars
    annotate_patches(True, ax.patches, '%d', 1.0, 22, dual_annotation=True)
    ax.text(0, 0, 'Only includes respondents responsible for housing costs.', horizontalalignment='left',
            verticalalignment='bottom', size='large', transform=ax.transAxes)
    legend = ax.legend(loc='upper right', shadow=True, fontsize='large')
    save_figure('%s_response.png' % field_name, freq_chart_dir)

def plot_title(title_line, fontsize=18, prefix=''):
    global plt
    title = '%s%s' % (prefix, title_line)
    plt.title(title, fontsize=fontsize)


print ('Defined. done.')

<hr>
<hr>
<hr>
<hr>
<hr>


<h2>Plot Generation</h2>

Run any of these sections independently, they are not cumulative

In [ ]:
for field in plotdf.columns:
    print(f'{field} {plotdf[field].value_counts()}')
    print('---')
print ('done')

In [ ]:
for field in plotdf.columns:
    plot_name = field
    if plot_name.startswith('Activity'):
        plot_name = 'Activity' + ' ' + plot_name[len('Activity'):]
    freq_dist_plot(field, plot_name)
print ('done')

In [ ]:
# function to draw stacked bar chart with %s and numbers in annotation
# using % for sizing of the stacked bar

from matplotlib import cm

# dusty rose, light tan , light teal from xkcd
stress_colors = np.array(
[[0xc0,0x73,0x7a,255],
 [0xfb,0xee,0xac,255.0],
 [0x90,0xe4,0xc1,255]
])
stress_colors /= 255.0

# orange pink #ff6f52
migration_colors = np.array(
[[0xff,0x6f,0x52,255],
 [0xd8,0xdc,0xd6,255.0],
 [0x98,0xef,0xf9,255]
])
migration_colors /= 255.0

def walk_df_by_columns(df):
    for col in df.columns:
        for index, row in df.iterrows():
            yield row[col]
            
def annotate_pct_patches_stacked(df_whole, patches, label_fontsize=14):
    """
    general routing to annotate patches
    
    patches - pass in ax.patches
    formatter: '%2.1f%%' for percent and '%d' for integer
    """
    global ax
    print(df_whole)
    assume_height_placement_offset = 52
    if len(patches)>10:
        # adjust for charts with many patches
        assume_height_placement_offset = assume_height_placement_offset / 2
    label_horiz_adj = label_fontsize * 2.0 / 3.0
    cum_value = 0
    for p in patches:
        small_bar_adjustment = 0
        if p.get_width() < 50:
            small_bar_adjustment = p.get_width()+12  # shift over the whole bar length so figure appears to right of bar
        cum_value += p.get_width()
        text_str = '%2.0f%%' % (p.get_width()*100)
        text_len_offset = (1.0*len(text_str))/2.0
        if p.get_width() > 0:  # only patches with width
            ax.annotate(text_str, (p.get_x() + p.get_width()/2.3, p.get_y()),
                        xytext=(-label_horiz_adj*text_len_offset + small_bar_adjustment, assume_height_placement_offset), 
                        fontsize=label_fontsize, textcoords='offset points')
            
    # dual annotation
    # show percentages as well
    assume_height_placement_offset = 15
    if len(patches)>10:
        # adjust for charts with many patches
        assume_height_placement_offset = assume_height_placement_offset / 2
    gen = walk_df_by_columns(df_whole)
    for cntr, p in enumerate(patches):
        small_bar_adjustment = 0
        if p.get_width() < 50:
            small_bar_adjustment = p.get_width()+12
        patch_val = next(gen) # grab zero or NaN as it may be even though patch may get skipped as no size
        if p.get_width() > 0:  # only patches with width
            text_str = '%d' % int(patch_val)
            text_len_offset = (1.0*len(text_str))/2.0
            ax.annotate(text_str, (p.get_x() + p.get_width()/2.3, p.get_y()),
                        xytext=(-label_horiz_adj*text_len_offset + small_bar_adjustment, 
                               assume_height_placement_offset), 
                        fontsize=label_fontsize-3, textcoords='offset points')

                    
def barh_stacked_number_pct(dfp, plot_title_str,  ylabel, legend_location,
                            colormap, ylabel_dollar_prefix=True, 
                            do_ylabel_range_annotation=True, responsible_disclaimer=True):
    '''
    Draw stacked bar chart based on %s per category
    But also annotate with absolute numbers
    
    dfp = data to print
    
    '''
    global ax
    params = {'legend.fontsize': 14,
              'legend.handlelength': 1}
    plt.rcParams.update(params)
    dfp_pct = dfp.iloc[:,:].apply(lambda y: y/y.sum(), axis=1)
    # maintain sort order from dfp
    #dfp_pct.sort_index(inplace=True)

    ax = dfp_pct.plot.barh(stacked=True,fontsize=14, color=colormap, figsize=(12,12)) 
    plot_title(plot_title_str, fontsize=18)
    if do_ylabel_range_annotation:
        ylabel_range_annotation(ax, ylabel_dollar_prefix)
    ax.set_ylabel(ylabel, fontsize=16)
    ax.set_xlabel('')
    ax.set_xticks([])

    # Legend
    legend = ax.legend(loc=legend_location, framealpha=0.4)
    print('annotating patches float')
    annotate_pct_patches_stacked(dfp, ax.patches) 
    # Label numbers on chart
    
    # not going to work - since we need absolute values:
    #annotate_patches_integer(False, ax.patches)
    
    if responsible_disclaimer:
        ax.text(0, 0, 'Only includes respondents responsible for housing costs.', horizontalalignment='left',
                verticalalignment='bottom', fontsize=12, transform=ax.transAxes)





In [ ]:
# Income v. Stress - Owners
dfc = plotdf.loc[(plotdf['IncomeCode1'] >=0) & (plotdf['StressCode']>=0) & (plotdf['OwnRent']=='Own'), 
                 ['IncomeCode1', 'StressCode', 'StressBand']]
dfc.IncomeCode1=dfc.IncomeCode1.astype('int32')
dfp = pd.pivot_table(dfc, 
                     values='StressCode',
                     index='IncomeCode1', columns='StressBand', aggfunc='count')
dfp=dfp[['2. High Stress', '1. Medium Stress', '0. Low Stress']]
dfp.sort_index(inplace=True)
barh_stacked_number_pct(dfp, 'Income v. Housing Stress (Owners)', 'Income', 'upper right', stress_colors )
save_figure('owner_income_stress_percent.png')   

In [ ]:
# Income v. Stress - Renters
dfc = plotdf.loc[(plotdf['IncomeCode1'] >=0) & (plotdf['StressCode']>=0) & (plotdf['OwnRent']=='Rent'),
                 ['IncomeCode1', 'StressCode', 'StressBand']]
dfc.IncomeCode1=dfc.IncomeCode1.astype('int32')
dfp = pd.pivot_table(dfc, 
                     values='StressCode',
                     index='IncomeCode1', columns='StressBand', aggfunc='count')


dfp=dfp[['2. High Stress', '1. Medium Stress', '0. Low Stress']]
dfp.sort_index(inplace=True)

barh_stacked_number_pct(dfp, 'Income v. Housing Stress (Renters)', 'Income', 'upper right', stress_colors )
save_figure('renter_income_stress_percent.png')  


In [ ]:
# Market Value v. Stress

for mvc in ['MarketValueCode','MarketValueCode1']:
    
    dfc = plotdf.loc[(plotdf[mvc] >=0) & (plotdf['StressCode']>=0) & (plotdf['OwnRent']=='Own'),
                     [mvc, 'StressCode', 'StressBand']]
    dfc[mvc]=dfc[mvc].astype('int32')

    dfp = pd.pivot_table(dfc,  values='StressCode',
                         index=mvc, columns='StressBand', aggfunc='count')
    dfp=dfp[['2. High Stress', '1. Medium Stress', '0. Low Stress']]

    dfp.sort_index(inplace=True)
    dfp_pct = dfp.iloc[:,:].apply(lambda y: y/y.sum(), axis=1)
    dfp_pct.sort_index(inplace=True)
    
    barh_stacked_number_pct(dfp, 'Home Value v. Housing Stress (Owners)','Home Value', 'upper right', stress_colors )
    save_figure('owner_%s_stress_percent.png' % mvc)
    


In [ ]:
# Presentation Plot
dfc = plotdf.loc[(plotdf['MarketValueCode'] >=0) & (plotdf['StressCode']>=0) & (plotdf['OwnRent']=='Own'),
                 ['MarketValueCode', 'StressCode', 'StressBand']]
dfc.MarketValueCode=dfc.MarketValueCode.astype('int32')

dfp = pd.pivot_table(dfc, 
                     values='StressCode',
                     index='MarketValueCode', columns='StressBand', aggfunc='count')
dfp=dfp[['2. High Stress', '1. Medium Stress', '0. Low Stress']]

dfp.sort_index(inplace=True)

#####################################

dfp_pct = dfp.iloc[:,:].apply(lambda y: y/y.sum(), axis=1)
dfp_pct.sort_index(inplace=True)
dfp_pct = dfp_pct['2. High Stress']
params = {'legend.fontsize': 14,
          'legend.handlelength': 1}
plt.rcParams.update(params)

ax = dfp_pct.plot.barh(stacked=True,  fontsize=16, color='xkcd:dusty rose', figsize=(12,12))  # df.plot.bar(title="Scores")
plot_title('High Housing Stress Reported (Homeowners)', fontsize=18, prefix='')
ylabel_range_annotation(ax)

ax.set_ylabel('Home Value', fontsize=16)
ax.set_xlabel('')
ax.set_xticks([])
annotate_patches_float(True, ax.patches, precision=0)

save_figure('owner_marketvalue_highstress_percent.png')

In [ ]:
# MonthlyHousingCost v. Stress - Owners
dfc = plotdf.loc[(plotdf['MonthlyHousingCost'] != 'ResponseError') & (plotdf['MonthlyHousingCost'].isna() == False) & (plotdf['StressCode']>=0) & (plotdf['OwnRent']=='Own'), 
                 ['MonthlyHousingCost', 'StressCode', 'StressBand']]
dfp = pd.pivot_table(dfc, 
                     values='StressCode',
                     index='MonthlyHousingCost', columns='StressBand', aggfunc='count')
dfp=dfp[['2. High Stress', '1. Medium Stress', '0. Low Stress']]


special_sort(dfp, 'MonthlyHousingCost')
#dfp.sort_index(inplace=True)

barh_stacked_number_pct(dfp, 'Monthly Housing Cost v. Housing Stress (Owners)', 'Monthly Housing Cost', 'upper right', stress_colors,
                       ylabel_dollar_prefix=True, do_ylabel_range_annotation=False)
save_figure('owner_monthlyhousing_stress_percent.png')   

In [ ]:
# MonthlyHousingCost v. Stress - Owners

dfc = plotdf.loc[(plotdf['PropertyTaxShare'].isna()==False) & (plotdf['StressCode'].isna()==False) & (plotdf['OwnRent']=='Own'), 
                 ['PropertyTaxShare', 'StressCode', 'StressBand']]
dfp = pd.pivot_table(dfc, 
                     values='StressCode',
                     index='PropertyTaxShare', columns='StressBand', aggfunc='count')
dfp=dfp[['2. High Stress', '1. Medium Stress', '0. Low Stress']]


special_sort(dfp, 'PropertyTaxShare')
#dfp.sort_index(inplace=True)

barh_stacked_number_pct(dfp, 'Monthly Housing Cost v. Housing Stress (Owners)', 'Property Tax Share', 'upper right', stress_colors,
                       ylabel_dollar_prefix=False, do_ylabel_range_annotation=False)
save_figure('owner_propertytaxshare_stress_percent.png')   

In [ ]:
dfc = plotdf.loc[(plotdf['OwnRent'] != '-1') & (plotdf['StressCode']>=0),
                 ['OwnRent', 'StressCode', 'StressBand']]
dfp = pd.pivot_table(dfc, 
                     values='StressCode',
                     index='OwnRent', columns='StressBand', aggfunc='count')
dfp=dfp[['2. High Stress', '1. Medium Stress', '0. Low Stress']]

dfp.sort_index(inplace=True)

##    
barh_stacked_number_pct(dfp, 'Own/Rent v. Housing Stress', 'Own or Rent', 'upper right', stress_colors )
save_figure('ownrent_stress_percent.png')

dfp_pct = dfp.iloc[:,:].apply(lambda y: y/y.sum(), axis=1)
dfp_pct.sort_index(inplace=True)
ax = dfp_pct.plot.barh(stacked=True,  fontsize=22, figsize=(12,12))  # df.plot.bar(title="Scores")
plot_title('Own/Rent v. Stress (response %)')
ax.set_ylabel('Own or Rent', fontsize=20)
ax.set_xlabel('')
ax.set_xticks([])
label_fontsize = 16
label_horiz_adj = label_fontsize * 2.0 / 3.0
legend = ax.legend(loc='lower right', shadow=True, fontsize='x-large')
annotate_patches_float(True, ax.patches)
ax.text(0, 0, 'Only includes respondents responsible for housing costs.', horizontalalignment='left',
        verticalalignment='bottom', size='large', transform=ax.transAxes)
save_figure('ownrent_stress_percent.png')    

###

ax = dfp.plot.barh(stacked=False, fontsize=20, figsize=(12,12))
plot_title('Own/Rent Value v. Stress (response counts)')
ax.set_ylabel('Own or Rent', fontsize=20)
ax.set_xlabel('Response Count', fontsize=20)

legend = ax.legend(loc='lower right', shadow=True, fontsize='x-large')
annotate_patches_integer(False, ax.patches)
ax.text(0, 0, 'Only includes respondents responsible for housing costs.', horizontalalignment='left',
        verticalalignment='bottom', size='large', transform=ax.transAxes)
save_figure('ownrent_stress_response.png')

In [ ]:
# Presentation Chart
dfc = plotdf.loc[(plotdf['OwnRent'] != '-1') & (plotdf['StressCode']>=0),
                 ['OwnRent', 'StressCode', 'StressBand']]
dfp = pd.pivot_table(dfc, 
                     values='StressCode',
                     index='OwnRent', columns='StressBand', aggfunc='count')
dfp=dfp[['2. High Stress', '1. Medium Stress', '0. Low Stress']]


dfp.sort_index(inplace=True)
print(dfp)

##

dfp_pct = dfp.iloc[:,:].apply(lambda y: y/y.sum(), axis=1)
dfp_pct.sort_index(inplace=True)

print(dfp_pct)
dfp_pct = dfp_pct['2. High Stress']
print(dfp_pct)

ax = dfp_pct.plot.barh(stacked=True,  fontsize=16,  color='xkcd:dusty rose', figsize=(12,12))  # df.plot.bar(title="Scores")
plot_title('High Housing Stress Reported', fontsize=18, prefix='')
ax.set_ylabel('')
ax.set_xlabel('')
ax.set_xticks([])
label_fontsize =18
label_horiz_adj = label_fontsize * 2.0 / 3.0
#legend = ax.legend(loc='upper right', shadow=True, fontsize='x-large')
annotate_patches_float(True, ax.patches, precision=0, label_fontsize=16)
#ax.text(0, 0, 'Only includes respondents responsible for housing costs.', horizontalalignment='left',
#        verticalalignment='bottom', size='large', transform=ax.transAxes)

save_figure('ownrent_highstress_percent.png')    
print('done')

In [ ]:
# Own/Rent v. Migration
dfc = plotdf.loc[(plotdf['OwnRent'] != '-1') & (plotdf['RelocationBand'].isna()==False),
                 ['OwnRent', 'RelocationBand', 'RelocationCode']]
dfp = pd.pivot_table(dfc, 
                     values='RelocationCode',
                     index='OwnRent', columns='RelocationBand', aggfunc='count')

dfp.sort_index(inplace=True)
dfp_pct = dfp.iloc[:,:].apply(lambda y: y/y.sum(), axis=1)
#dfp_pct.sort_index(inplace=True)
barh_stacked_number_pct(dfp, 'Owning and Renting v. Migration','Own / Rent', 'upper right', migration_colors,
                        ylabel_dollar_prefix=False, 
                            do_ylabel_range_annotation=False)
save_figure('own_rent_migration_percent.png')    

In [ ]:
for mvc in ['MarketValueCode','MarketValueCode1']:
    dfc = plotdf.loc[(plotdf[mvc] != -1) & (plotdf['RelocationBand'].isna()==False) & (plotdf['OwnRent'] == 'Own'),
                     [mvc, 'RelocationBand', 'RelocationCode']]
    dfc[mvc]=dfc[mvc].astype('int32')

    dfp = pd.pivot_table(dfc, 
                         values='RelocationCode',
                         index=mvc, columns='RelocationBand', aggfunc='count')

    dfp.sort_index(inplace=True)
    dfp_pct = dfp.iloc[:,:].apply(lambda y: y/y.sum(), axis=1)
    #dfp_pct.sort_index(inplace=True)
    barh_stacked_number_pct(dfp, 'Home Value v. Migration (Owners)','Home Value', 'upper right', migration_colors)
    save_figure('owner_%s_migration_percent.png' % mvc)    

In [ ]:
print(plotdf.IncomeCode1.value_counts())
dfc = plotdf.loc[(plotdf['IncomeCode1'] != -1) & (plotdf['RelocationBand'].isna()==False) & (plotdf['OwnRent'] == 'Own'),
                 ['IncomeCode1', 'RelocationBand', 'RelocationCode']]
dfc.IncomeCode1=dfc.IncomeCode1.astype('int32')

dfp = pd.pivot_table(dfc, 
                     values='RelocationCode',
                     index='IncomeCode1', columns='RelocationBand', aggfunc='count')

dfp.sort_index(inplace=True)

##

dfp_pct = dfp.iloc[:,:].apply(lambda y: y/y.sum(), axis=1)
dfp_pct.sort_index(inplace=True)
barh_stacked_number_pct(dfp, 'Income v. Migration (Owners)', 'Income', 'upper right', migration_colors)
save_figure('owner_income_migration_percent.png')


In [ ]:
dfc = plotdf.loc[(plotdf['IncomeCode1'].isna()==False) & (plotdf['RelocationBand'].isna()==False) & (plotdf['OwnRent'] == 'Rent'),
                 ['IncomeCode1', 'RelocationBand', 'RelocationCode']]
dfc.IncomeCode1=dfc.IncomeCode1.astype('int32')

dfp = pd.pivot_table(dfc, 
                     values='RelocationCode',
                     index='IncomeCode1', columns='RelocationBand', aggfunc='count')

dfp.sort_index(inplace=True)
dfp_pct = dfp.iloc[:,:].apply(lambda y: y/y.sum(), axis=1)
dfp_pct.sort_index(inplace=True)
barh_stacked_number_pct(dfp, 'Income v. Migration (Renters)', 'Income', 'upper right', migration_colors)
save_figure('renter_income_migration_percent.png')

In [ ]:
dfc = plotdf.loc[(plotdf['MonthlyRentCode'] != -1) & (plotdf['StressCode']>=0) & (plotdf['OwnRent'] == 'Rent'),
                 ['MonthlyRentCode', 'StressCode', 'StressBand']]
dfc.MonthlyRentCode=dfc.MonthlyRentCode.astype('int32')
dfp = pd.pivot_table(dfc, 
                     values='StressCode',
                     index='MonthlyRentCode', columns='StressBand', aggfunc='count')
dfp=dfp[['2. High Stress', '1. Medium Stress', '0. Low Stress']]

dfp.sort_index(inplace=True)
dfp_pct = dfp.iloc[:,:].apply(lambda y: y/y.sum(), axis=1)
dfp_pct.sort_index(inplace=True)
barh_stacked_number_pct(dfp, 'Rent v. Housing Stress (Renters)', 'Monthly Rent', 'upper right', stress_colors)
save_figure('renter_rent_stress_percent.png')


In [ ]:
dfc = plotdf.loc[(plotdf['MonthlyRentCode'] != -1) & (plotdf['RelocationBand'].isna()==False) & (plotdf['OwnRent'] == 'Rent'),
                 ['MonthlyRentCode', 'RelocationBand', 'RelocationCode']]
dfc.MonthlyRentCode=dfc.MonthlyRentCode.astype('int32')
dfp = pd.pivot_table(dfc, 
                     values='RelocationCode',
                     index='MonthlyRentCode', columns='RelocationBand', aggfunc='count')

dfp.sort_index(inplace=True)

##

dfp_pct = dfp.iloc[:,:].apply(lambda y: y/y.sum(), axis=1)
dfp_pct.sort_index(inplace=True)
barh_stacked_number_pct(dfp, 'Rent v. Migration (Renters)', 'Monthly Rent', 'upper right', migration_colors)
save_figure('renter_rent_migration_percent.png')

In [ ]:
for ac in ['AgeCode','AgeCode1']:

    dfc = plotdf.loc[(plotdf[ac].isna()==False) & (plotdf['RelocationBand'].isna()==False) & (plotdf['OwnRent'] == 'Own'),
                     [ac, 'RelocationBand', 'RelocationCode']]
    dfc[ac]=dfc[ac].astype('int32')
    dfp = pd.pivot_table(dfc, 
                         values='RelocationCode',
                         index=ac, columns='RelocationBand', aggfunc='count')

    dfp.sort_index(inplace=True)
    dfp_pct = dfp.iloc[:,:].apply(lambda y: y/y.sum(), axis=1)
    dfp_pct.sort_index(inplace=True)
    barh_stacked_number_pct(dfp, 'Age v. Migration (Owners)', 'Age', 'upper right', migration_colors, ylabel_dollar_prefix=False)
    save_figure('owner_%s_migration_percent.png' % ac)

In [ ]:
dfc = plotdf.loc[(plotdf['AgeCode1'].isna()==False) & (plotdf['RelocationBand'].isna()==False) & (plotdf['OwnRent'] == 'Rent'),
                 ['AgeCode1', 'RelocationBand', 'RelocationCode']]
dfc.AgeCode1=dfc.AgeCode1.astype('int32')

dfp = pd.pivot_table(dfc, 
                     values='RelocationCode',
                     index='AgeCode1', columns='RelocationBand', aggfunc='count')

dfp.sort_index(inplace=True)

##

dfp_pct = dfp.iloc[:,:].apply(lambda y: y/y.sum(), axis=1)
dfp_pct.sort_index(inplace=True)
barh_stacked_number_pct(dfp, 'Age v. Migration (Renters)', 'Age', 'upper right', migration_colors, ylabel_dollar_prefix=False)
save_figure('renter_age_migration_percent.png')


In [ ]:
dfc = plotdf.loc[(plotdf['RelocationBand'].isna()==False) & (plotdf['StressCode'].isna()==False) & (plotdf['OwnRent']=='Own'),
                 ['RelocationBand', 'StressCode', 'StressCodeDescriptive']]
dfp = pd.pivot_table(dfc, values='StressCode',
                     index='StressCodeDescriptive', columns='RelocationBand', aggfunc='count')
dfp.sort_index(inplace=True)

###

dfp_pct = dfp.iloc[:,:].apply(lambda y: y/y.sum(), axis=1)
dfp_pct.sort_index(inplace=True)
barh_stacked_number_pct(dfp, 'Housing Stress v. Migration (Owners)', 'Housing Stress', 'upper right', migration_colors,
                        ylabel_dollar_prefix=False, do_ylabel_range_annotation=False)
save_figure('owner_stress_migration_percent.png')


In [ ]:
# Migration v. Tenure
for ownrent, ownrentlabel in [('Own', 'Owners'), ('Rent','Renters')]:
    dfc = plotdf.loc[(plotdf['RelocationBand'].isna()==False) & (plotdf['StressCode'].isna()==False) & (plotdf['OwnRent']==ownrent),
                     ['RelocationBand', 'Tenure', 'TenureStat']]
    #print dfc
    dfp = pd.pivot_table(dfc, values='TenureStat',
                         index='Tenure', columns='RelocationBand', aggfunc='count')
    dfp.sort_index(inplace=True)
    dfp_pct = dfp.iloc[:,:].apply(lambda y: y/y.sum(), axis=1)
    dfp_pct.sort_index(inplace=True)
    barh_stacked_number_pct(dfp, 'Time in Lexington v. Migration (%s)' % ownrentlabel, 'Time in Lexington', 'upper right', migration_colors,
                            ylabel_dollar_prefix=False, do_ylabel_range_annotation=False)
    save_figure('%sers_tenure_migration_percent.png' % ownrent)


In [ ]:
# Migration v. Monthly Housing Cost
for ownrent, ownrentlabel in [('Own', 'Owners')]:
    dfc = plotdf.loc[(plotdf['MonthlyHousingCost'] != 'ResponseError') &(plotdf['RelocationBand'].isna()==False) & 
                     (plotdf['StressCode'].isna()==False) & (plotdf['OwnRent']==ownrent),
                     ['RelocationBand', 'MonthlyHousingCost', 'MonthlyHousingStat']]
   # print dfc
    dfp = pd.pivot_table(dfc, values='MonthlyHousingStat',
                         index='MonthlyHousingCost', columns='RelocationBand', aggfunc='count')
    # dfp.sort_index(inplace=True)
    special_sort(dfp, 'MonthlyHousingCost')    
    
    
    dfp_pct = dfp.iloc[:,:].apply(lambda y: y/y.sum(), axis=1)
    dfp_pct.sort_index(inplace=True)
    barh_stacked_number_pct(dfp, 'Monthly Housing Cost v. Migration (%s)' % ownrentlabel, 'Monthly Housing Cost', 'upper right', migration_colors,
                            ylabel_dollar_prefix=False, do_ylabel_range_annotation=False)
    save_figure('%sers_monthlyhousingcost_migration_percent.png' % ownrent)


In [ ]:
# Migration v. Property Tax Share
for ownrent, ownrentlabel in [('Own', 'Owners')]:
    dfc = plotdf.loc[(plotdf['RelocationBand'].isna()==False) & (plotdf['StressCode'].isna()==False) & (plotdf['OwnRent']==ownrent),
                     ['RelocationBand', 'PropertyTaxShare', 'PropertyTaxShareStat']]
   # print dfc
    dfp = pd.pivot_table(dfc, values='PropertyTaxShareStat',
                         index='PropertyTaxShare', columns='RelocationBand', aggfunc='count')
    # dfp.sort_index(inplace=True)
    special_sort(dfp, 'PropertyTaxShare')    
    
    
    dfp_pct = dfp.iloc[:,:].apply(lambda y: y/y.sum(), axis=1)
    dfp_pct.sort_index(inplace=True)
    barh_stacked_number_pct(dfp, 'Property Tax Share v. Migration (%s)' % ownrentlabel, 'Property Tax Share', 'upper right', migration_colors,
                            ylabel_dollar_prefix=False, do_ylabel_range_annotation=False)
    save_figure('%sers_propertytaxshare_migration_percent.png' % ownrent)


In [ ]:
# for presentation
dfc = plotdf.loc[(plotdf['RelocationBand'].isna()==False) & (plotdf['StressCode'].isna()==False) & (plotdf['OwnRent']=='Own'),
                 ['RelocationBand', 'StressCode', 'StressCodeDescriptive']]
dfp = pd.pivot_table(dfc, values='StressCode',
                     index='StressCodeDescriptive', columns='RelocationBand', aggfunc='count')
dfp.sort_index(inplace=True)

###

dfp_pct = dfp.iloc[:,:].apply(lambda y: y/y.sum(), axis=1)
dfp_pct.sort_index(inplace=True)
dfp_pct = dfp_pct['Likely']

ax = dfp_pct.plot.barh(stacked=True,  fontsize=22, color='orange', figsize=(12,12))  # df.plot.bar(title="Scores")
plot_title('Housing Stress v. Predicted Migration (Homeowners)', fontsize=26, prefix='')
#ylabel_range_annotation(ax, False)
ax.set_ylabel('Stress', fontsize=28)
ax.set_xlabel('')
ax.set_xticks([])
label_fontsize = 16
label_horiz_adj = label_fontsize * 2.0 / 3.0

annotate_patches_float(True, ax.patches, precision=0)
save_figure('owner_highstress_migration_percent.png')
    

In [ ]:
dfc = plotdf.loc[(plotdf['RelocationBand'].isna()==False) & (plotdf['StressCode'].isna()==False) & (plotdf['OwnRent']=='Rent'),
                 ['RelocationBand', 'StressCode', 'StressCodeDescriptive']]
dfp = pd.pivot_table(dfc, values='StressCode',
                     index='StressCodeDescriptive', columns='RelocationBand', aggfunc='count')

dfp.sort_index(inplace=True)

###

dfp_pct = dfp.iloc[:,:].apply(lambda y: y/y.sum(), axis=1)
dfp_pct.sort_index(inplace=True)
barh_stacked_number_pct(dfp, 'Housing Stress v. Migration (Renters)', 'Housing Stress', 'upper right', migration_colors,
                        ylabel_dollar_prefix=False, do_ylabel_range_annotation=False)
save_figure('renter_stress_migration_percent.png')


In [ ]:
# Age v. Housing Stress - Renters and Owners, Both Age Code Sets
for housing_status in [['Own', 'Owners'], ['Rent', 'Renters']]:
    ownrent_filter, ownrent_descriptor = housing_status
    ac = 'Tenure'
    dfc = plotdf.loc[(plotdf[ac].isna()==False) & (plotdf['StressCode'].isna()==False) & (plotdf['OwnRent']==ownrent_filter),
                     [ac, 'StressCode', 'StressBand']]
    dfp = pd.pivot_table(dfc, values='StressCode',
                         index=ac, columns='StressBand', aggfunc='count')
    dfp=dfp[['2. High Stress', '1. Medium Stress', '0. Low Stress']]
    dfp.sort_index(inplace=True)
    dfp_pct = dfp.iloc[:,:].apply(lambda y: y/y.sum(), axis=1)
    dfp_pct.sort_index(inplace=True)
    barh_stacked_number_pct(dfp, 'Tenure v. Housing Stress (%s)' % ownrent_descriptor, 
                            'Tenure', 'upper right', stress_colors,  ylabel_dollar_prefix=False,
                       do_ylabel_range_annotation=False)
    save_figure('%s_%s_stress_percent.png' % (ownrent_filter, ac))

In [ ]:
# Age v. Housing Stress - Renters and Owners, Both Age Code Sets
for ac in ['AgeCode1', 'AgeCode']:
    for housing_status in [['Own', 'Owners'], ['Rent', 'Renters']]:
        ownrent_filter, ownrent_descriptor = housing_status

        dfc = plotdf.loc[(plotdf[ac] >=0) & (plotdf['StressCode']>=0) & (plotdf['OwnRent']==ownrent_filter),
                         [ac, 'StressCode', 'StressBand']]
        dfc[ac]=dfc[ac].astype('int32')
        
        dfp = pd.pivot_table(dfc, values='StressCode',
                             index=ac, columns='StressBand', aggfunc='count')
        dfp=dfp[['2. High Stress', '1. Medium Stress', '0. Low Stress']]
        dfp.sort_index(inplace=True)
        dfp_pct = dfp.iloc[:,:].apply(lambda y: y/y.sum(), axis=1)
        dfp_pct.sort_index(inplace=True)
        barh_stacked_number_pct(dfp, 'Age v. Housing Stress (%s)' % ownrent_descriptor, 
                                'Age', 'upper right', stress_colors, ylabel_dollar_prefix=False)
        save_figure('%s_%s_stress_percent.png' % (ownrent_filter, ac))

In [ ]:
for housing_status in [['Own', 'Owners']]:
    ownrent_filter, ownrent_descriptor = housing_status

    dfc = plotdf.loc[(plotdf['AgeCode1'] >=0) & (plotdf['StressCode']>=0) & (plotdf['OwnRent']==ownrent_filter),
                     ['AgeCode1', 'StressCode', 'StressBand']]
    dfc.AgeCode1=dfc.AgeCode1.astype('int32')
    
    dfp = pd.pivot_table(dfc, values='StressCode',
                         index='AgeCode1', columns='StressBand', aggfunc='count')
    dfp=dfp[['2. High Stress', '1. Medium Stress', '0. Low Stress']]

    dfp.sort_index(inplace=True)

    ###

    dfp_pct = dfp.iloc[:,:].apply(lambda y: y/y.sum(), axis=1)
    dfp_pct.sort_index(inplace=True)
    dfp_pct = dfp_pct['2. High Stress']
    ax = dfp_pct.plot.barh(stacked=True,  fontsize=22, color='orange', figsize=(12,12))  # df.plot.bar(title="Scores")
    plot_title('High Housing Stress Reported (Owners)', fontsize=30, prefix='')
    ylabel_range_annotation(ax, False)
    ax.set_ylabel('Age', fontsize=28)
    ax.set_xlabel('')
    ax.set_xticks([])
    label_fontsize = 16
    label_horiz_adj = label_fontsize * 2.0 / 3.0

    annotate_patches_float(True, ax.patches, precision=0)
    save_figure('%s_age_highstress_percent.png' % ownrent_filter)


In [ ]:
# stress v. income & age
# attempt to do two variables

#'$100,000-$149,999':100000,
#'$150,000-$199,999':150000,
#'$200,000 and over':200000,
#'$35,000-$49,999':0,
#'$50,000-$74,999':50000,
#'$75,000-$99,000':75000,
#'Less than $35,000':0,

# not enough cases so use IncomeCode2
#        '$100,000-$149,999':100000,
#'$150,000-$199,999':100000,
#'$200,000 and over':200000,
#'$35,000-$49,999':0,
#'$50,000-$74,999':0,
#'$75,000-$99,000':0,
#'Less than $35,000':0,

# not enough data to break out this way for renters        
for housing_status in [['Own', 'Owners']]:
    for income_code, income_desc in [
        [0, 'Less than \$99,999'],
        [100000, '\$100,000-\$199,999'],
        [200000, '\$200,000 and over']]:
        
    
        ownrent_filter, ownrent_descriptor = housing_status

        dfc = plotdf.loc[(plotdf['AgeCode1'] >=0) & (plotdf['StressCode']>=0) & (plotdf['IncomeCode2']==income_code) & (plotdf['OwnRent']==ownrent_filter),
                         ['AgeCode1', 'StressCode', 'StressBand']]
        dfc.AgeCode1=dfc.AgeCode1.astype('int32')
        dfp = pd.pivot_table(dfc, values='StressCode',
                             index='AgeCode1', columns='StressBand', aggfunc='count')
        dfp=dfp[['2. High Stress', '1. Medium Stress', '0. Low Stress']]

        dfp.sort_index(inplace=True)

        ###

        dfp_pct = dfp.iloc[:,:].apply(lambda y: y/y.sum(), axis=1)
        dfp_pct.sort_index(inplace=True)
        ax = dfp_pct.plot.barh(stacked=True,  fontsize=22, figsize=(12,12))  # df.plot.bar(title="Scores")
        plot_title('Income Range %s: Age v. Stress (%s)' % (income_desc, ownrent_descriptor))
        ylabel_range_annotation(ax, False)
        ax.set_ylabel('Age', fontsize=20)
        ax.set_xlabel('')
        ax.set_xticks([])
        label_fontsize = 16
        label_horiz_adj = label_fontsize * 2.0 / 3.0

        annotate_patches_float(True, ax.patches)
        legend = ax.legend(loc='upper right', shadow=True, fontsize='x-large')
        ax.text(0, 0, 'Only includes respondents responsible for housing costs.', horizontalalignment='left',
                verticalalignment='bottom', size='large', transform=ax.transAxes)
        save_figure('%s_income_%d_age_stress_percent.png' % (ownrent_filter, income_code))

        ###

        ax = dfp.plot.barh(stacked=False, fontsize=20, figsize=(12,12))
        plot_title('Income Range %s: Age v. Stress (%s)' % (income_desc, ownrent_descriptor))
        ylabel_range_annotation(ax, False)
        ax.set_ylabel('Age', fontsize=20)
        ax.set_xlabel('Response Count', fontsize=20)
        legend = ax.legend(loc='lower right', shadow=True, fontsize='x-large')
        annotate_patches_integer(False, ax.patches)
        ax.text(0, 0, 'Only includes respondents responsible for housing costs.', horizontalalignment='left',
                verticalalignment='bottom', size='large', transform=ax.transAxes)
        save_figure('%s_income_%d_age_stress_response.png' % (ownrent_filter, income_code))

In [ ]:
dfc = plotdf.loc[(plotdf['MeansTestedApprox'] != -1) & (plotdf['StressCode']>=0) & (plotdf['OwnRent']=='Own'),
                 ['MeansTestedApprox', 'StressCode', 'StressBand']]
dfp = pd.pivot_table(dfc, 
                     values='StressCode',
                     index='MeansTestedApprox', columns='StressBand', aggfunc='count')
dfp=dfp[['2. High Stress', '1. Medium Stress', '0. Low Stress']]

dfp.sort_index(inplace=True)

##

dfp_pct = dfp.iloc[:,:].apply(lambda y: y/y.sum(), axis=1)
dfp_pct.sort_index(inplace=True)
barh_stacked_number_pct(dfp, 'Means Tested Approx Qualified v. Housing Stress (Owners)', 
                        'Means Tested Approx Qulalified', 'upper right', stress_colors, ylabel_dollar_prefix=False,
                       do_ylabel_range_annotation=False)
save_figure('means_tested_stress_percent.png')    


In [ ]:
dfc = plotdf.loc[(plotdf['MeansTestedApprox'] != -1) & (plotdf['RelocationCode']>=0) & (plotdf['OwnRent']=='Own'),
                 ['MeansTestedApprox', 'RelocationCode', 'RelocationBand']]
dfp = pd.pivot_table(dfc, 
                     values='RelocationCode',
                     index='MeansTestedApprox', columns='RelocationBand', aggfunc='count')

dfp.sort_index(inplace=True)

##

dfp_pct = dfp.iloc[:,:].apply(lambda y: y/y.sum(), axis=1)
dfp_pct.sort_index(inplace=True)
barh_stacked_number_pct(dfp, 'Means Tested Approx Qualified v. Migration (Owers)', 
                        'Means Tested Approx Qulalified', 'upper right', migration_colors, ylabel_dollar_prefix=False,
                       do_ylabel_range_annotation=False)
save_figure('means_tested_own_migration_percent.png')    

In [ ]:
dfc = plotdf.loc[(plotdf['MeansTestedApprox'] != -1) & (plotdf['StressCode']>=0),
                 ['MeansTestedApprox', 'StressCode', 'StressBand']]
dfp = pd.pivot_table(dfc, 
                     values='StressCode',
                     index='MeansTestedApprox', columns='StressBand', aggfunc='count')
dfp=dfp[['2. High Stress', '1. Medium Stress', '0. Low Stress']]

dfp.sort_index(inplace=True)

##

dfp_pct = dfp.iloc[:,:].apply(lambda y: y/y.sum(), axis=1)
dfp_pct.sort_index(inplace=True)
dfp_pct = dfp_pct['2. High Stress']
ax = dfp_pct.plot.barh(stacked=True,  fontsize=32, color='orange', figsize=(12,12))  # df.plot.bar(title="Scores")
plot_title('High Housing Stress Reported (Homeowners)',fontsize=32, prefix='')

ax.set_ylabel('Means Tested Qualified (Approximately)', fontsize=28)
ax.set_xlabel('')
ax.set_xticks([])
label_fontsize = 22
label_horiz_adj = label_fontsize * 2.0 / 3.0
#legend = ax.legend(loc='lower right', shadow=True, fontsize='x-large')
annotate_patches_float(True, ax.patches, precision=0, label_fontsize=30)
save_figure('means_tested_highstress_percent.png')    


In [ ]:
# means-tested v. current use of LPS
print(plotdf.ResponsibleCode.value_counts())

dfc = plotdf.loc[(plotdf['MeansTestedApprox'].isna()==False) & (plotdf['ActivityLPS'].isna()==False),
                 ['MeansTestedApprox', 'ResponsibleCode', 'ActivityLPS']]


dfp = pd.pivot_table(dfc, 
                     values='ResponsibleCode',
                     index='MeansTestedApprox', columns='ActivityLPS', aggfunc='count')

dfp=dfp[['Yes', 'No']]

dfp.sort_index(inplace=True)

##

dfp_pct = dfp.iloc[:,:].apply(lambda y: y/y.sum(), axis=1)
dfp_pct.sort_index(inplace=True)
ax = dfp_pct.plot.barh(stacked=True,  fontsize=22, figsize=(12,12))  # df.plot.bar(title="Scores")
plot_title('Means Tested Approx Qualified v. Activity LPS (response %)')
ax.set_ylabel('Means Tested Approx Qualified', fontsize=20)
ax.set_xlabel('')
ax.set_xticks([])
label_fontsize = 16
label_horiz_adj = label_fontsize * 2.0 / 3.0
legend = ax.legend(loc='lower right', shadow=True, fontsize='x-large')
annotate_patches_float(True, ax.patches)
ax.text(0, 0, 'Only includes respondents responsible for housing costs.', horizontalalignment='left',
        verticalalignment='bottom', size='large', transform=ax.transAxes)
    
    
#plt.text(0, 1, 'Only includes respondents responsible for housing costs.', ha='left', wrap=True)
save_figure('means_tested_activity_lps_percent.png')    

###

ax = dfp.plot.barh(stacked=False, fontsize=20, figsize=(12,12))
plot_title('Means Tested Approx Qualified  v. Activity LPS (response counts)')
ax.set_ylabel('Means Tested Approx Qualified', fontsize=20)
ax.set_xlabel('Response Count', fontsize=20)

legend = ax.legend(loc='lower right', shadow=True, fontsize='x-large')
annotate_patches_integer(False, ax.patches)
ax.text(0, 0, 'Only includes respondents responsible for housing costs.', horizontalalignment='left',
        verticalalignment='bottom', size='large', transform=ax.transAxes)

save_figure('means_tested_activity_lps_response.png')

In [ ]:
dfc = plotdf.loc[(plotdf['MeansTestedApprox'].isna()==False) & (plotdf['ActivityLPS'].isna()==False),
                 ['MeansTestedApprox', 'ResponsibleCode', 'ActivityLPS']]
dfp = pd.pivot_table(dfc, 
                     values='ResponsibleCode',
                     index='MeansTestedApprox', columns='ActivityLPS', aggfunc='count')

dfp=dfp[['Yes', 'No']]
dfp.sort_index(inplace=True)
dfp_pct = dfp.iloc[:,:].apply(lambda y: y/y.sum(), axis=1)
dfp_pct.sort_index(inplace=True)
dfp_pct = dfp_pct['Yes']

ax = dfp_pct.plot.barh(stacked=True,  fontsize=22, color='orange', figsize=(12,12))  # df.plot.bar(title="Scores")
plot_title('Current Use of Lexington Public Schools (Homeowners)', fontsize=28, prefix='')
ax.set_ylabel('Means Tested Qualified (Approximately)', fontsize=24)
ax.set_xlabel('')
ax.set_xticks([])
label_fontsize = 16
label_horiz_adj = label_fontsize * 2.0 / 3.0
annotate_patches_float(True, ax.patches, precision=0, label_fontsize=30)
    
    
#plt.text(0, 1, 'Only includes respondents responsible for housing costs.', ha='left', wrap=True)
save_figure('means_tested_activity_lpsuse_percent.png')    

In [ ]:
MAX_CHARTS = 100
activity_list = ['ActivityMinutemanBikeway', 'ActivityCary', 'ActivityComCenter', 'ActivityConservation', 
                 'ActivityField', 'ActivityFirePolice', 'ActivityGolf', 'ActivityLPS', 'ActivityLexpress', 
                 'ActivityPlayground', 'ActivitySenior', 'ActivitySwim', 'ActivityTown']

for cntr, activity_field in enumerate(activity_list):
    
    if cntr >= MAX_CHARTS:
        break
    
    for housing_status in [['Own', 'Owners'], ['Rent', 'Renters']]:
        ownrent_filter, ownrent_descriptor = housing_status
    

        dfc = plotdf.loc[(plotdf[activity_field].isna()==False) & (plotdf['RelocationBand'].isna()==False) & (plotdf['OwnRent'] == ownrent_filter),
                         [activity_field, 'RelocationBand', 'RelocationCode']]
        dfp = pd.pivot_table(dfc, 
                             values='RelocationCode',
                             index=activity_field, columns='RelocationBand', aggfunc='count')
        dfp.sort_index(inplace=True)

        ##

        dfp_pct = dfp.iloc[:,:].apply(lambda y: y/y.sum(), axis=1)
        dfp_pct.sort_index(inplace=True)
        ax = dfp_pct.plot.barh(stacked=True,  fontsize=22, figsize=(12,12))  # df.plot.bar(title="Scores")
        plot_title('%s v. Migration (%s)' % (activity_field, ownrent_descriptor))
#        handles, labels = ax.get_legend_handles_labels()
        
        legend = ax.legend(loc='upper right', shadow=True, fontsize='x-large')

        ax.set_ylabel(activity_field, fontsize=20)
        ax.set_xlabel('')
        ax.set_xticks([])
        label_fontsize = 16
        label_horiz_adj = label_fontsize * 2.0 / 3.0
        annotate_patches_float(True, ax.patches)
        ax.text(0, 0, 'Only includes respondents responsible for housing costs.', horizontalalignment='left',
                verticalalignment='bottom', size='large', transform=ax.transAxes)
        save_figure('%s_%s_migration_percent.png' % (ownrent_filter, activity_field))

        #

        # Horizontal bar chart
        ax = dfp.plot.barh(stacked=False, fontsize=20, figsize=(12,12))
        plot_title('%s v. Migration (%s)' % (activity_field, ownrent_descriptor))
        ax.set_ylabel(activity_field, fontsize=20)
        ax.set_xlabel('Response Count', fontsize=20)
        legend = ax.legend(loc='lower right', shadow=True, fontsize='x-large')
        annotate_patches_integer(False, ax.patches)
        ax.text(0, 0, 'Only includes respondents responsible for housing costs.', horizontalalignment='left',
                verticalalignment='bottom', size='large', transform=ax.transAxes)
        save_figure('%s_%s_migration_response.png'% (ownrent_filter, activity_field))
    

In [ ]:
import glob
from fpdf import FPDF


#chart_dir = 'E:\\LexingtonSurvey\\charts\\relationships'
#freq_chart_dir = 'E:\\LexingtonSurvey\\charts\\frequencies'
file_list1 = glob.glob(os.path.join(freq_chart_dir, '*.png'))
print(file_list1)
                      
file_list2 = glob.glob(os.path.join(chart_dir, '*.png'))
print(file_list2)
imagelist = file_list2 + file_list1

pdf = FPDF()
pdf.set_font('Arial', '', 14)
pdf.add_page()
pdf.cell(w = 140, h = 20, txt = 'Lexington Survey.  Preliminary Result Packet')
pdf.set_font('Arial', '', 8)
pdf.ln(8)
pdf.cell(w = 140, h = 20, txt = 'Page. Chart Name')
pdf.ln(5)
# print list of images
for cntr, image in enumerate(imagelist):
    page_num = int(cntr / 4) + 3
    
    image_name = image[:-4]
    image_name = image_name[::-1]
    image_name = image_name[:image_name.find('\\')]
    image_name = image_name[::-1]
    pdf.cell(w = 140, h = 16, txt = '%d.  %s' % (page_num, image_name))
    pdf.ln(3)
    print('+')

for cntr, image in enumerate(imagelist):
    if cntr >= 3000:
        break
    if cntr % 4 == 0:
        pdf.add_page()
        page_num = int(cntr / 4) + 3
        pdf.cell(w = 140, h = 20, txt = 'Page %d' % page_num)
    x, y, w, h = (cntr%2)*105+5, ((cntr/2)%2) * 140 + 30, 100, 120
    pdf.image(image,x,y,w,h)
    print('.')
pdf.output('E:\\LexingtonSurvey\\charts\\chart_summary.pdf', "F")
print('done')

In [ ]:
# What Does the Survey Tell Us?
import re
filename = 'E:\\LexingtonSurvey\\whatdoesthesurveytellus.txt'
#chart_dir = 'E:\\LexingtonSurvey\\charts\\relationships'


pattern = '\[.*\]' 
prog = re.compile(pattern, re.DOTALL | re.UNICODE)

        
pdf = FPDF()
pdf.add_page()
pdf.set_font('Arial', '', 16)
pdf.cell(w = 140, h = 20, txt = 'What Does the Survey Tell Us?')
pdf.ln(6)
pdf.cell(w = 140, h = 20, txt = 'Preliminary Discussion (11/8/2018)')
pdf.ln(14)
pdf.set_font('Arial', '', 10)

with open(filename, 'r') as fh:
    while True:
        report_txt = fh.readline()
        if report_txt == '':
            break
        m = prog.match(report_txt)
        if m:
            report_txt = 'image: %s' % m.group(0)
            print report_txt
            img_filename = os.path.join(chart_dir, m.group(0)[1:-1])
            width, height = 90, 110
            pdf.image(img_filename, w=width, h=height)
            #pdf.cell( 40, 40, pdf.InlineImage(img_filename, pdf.GetX(), pdf.GetY(), 100), 0, 0, 'L', false );
        else:
            pdf.multi_cell(w = 180, h = 4, txt = report_txt)

        pdf.ln(6)   

pdf.output('E:\\LexingtonSurvey\\WhatDoesTheSurveyTellUs.pdf', "F")

print 'done'




<HR><HR><HR><HR><HR><HR><HR>

In [ ]:
raise 'stop here'

In [ ]:

ax = dfc['IncomeCode'].value_counts().plot(kind='barh', figsize=(10,7),
                                                 color="slateblue", fontsize=13);
ax.set_alpha(0.8)
ax.set_title("Income Code Distribution", fontsize=22)
ax.set_xlabel("Frequency", fontsize=18);
#ax.set_xticks([0, 5])

# set individual bar lables using above list
for i in ax.patches:
    # get_width pulls left or right; get_y pushes up or down
    ax.text(i.get_width()+.1, i.get_y()+.31, \
            str(round((i.get_width()), 2)), fontsize=15, color='dimgrey')

# invert for largest on top 
ax.invert_yaxis()
plt.show()

In [ ]:
print dfp[2].value_counts()

ax = dfp[2].value_counts().plot(kind='barh', figsize=(10,7),
                                                 color="slateblue", fontsize=13);
ax.set_alpha(0.8)
ax.set_title("Income Code Distribution", fontsize=22)
ax.set_xlabel("Frequency", fontsize=18);
#ax.set_xticks([0, 5])

# set individual bar lables using above list
for i in ax.patches:
    # get_width pulls left or right; get_y pushes up or down
    ax.text(i.get_width()+.1, i.get_y()+.31, \
            str(round((i.get_width()), 2)), fontsize=15, color='dimgrey')

# invert for largest on top 
ax.invert_yaxis()
plt.show()

<H3>ROC Curve Testing</h3>


In [ ]:
def plot_roc(title, fpr, tpr):
    lw = 3
    plt.rcParams.update({'font.size': 20})
    plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve')
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    fig = matplotlib.pyplot.gcf()
    fig.set_size_inches(12,12)

    ax = plt.gca()
    vals = ax.get_yticks()
    ax.set_yticklabels(['{:,.0%}'.format(x) for x in vals])
    vals = ax.get_xticks()
    ax.set_xticklabels(['{:,.0%}'.format(x) for x in vals])
    # do not show so caller can annotate

<h3>Write ROC and Precision-Recall curves automatically</h3>

In [ ]:
from sklearn.utils.fixes import signature
import matplotlib.ticker as mticker

def get_sequence(field_name):
    if field_name == 'Tenure':
        seq = ['moved in 2015 or later', 'moved in 2010-2014', 'moved in 2000-2009', 'moved in 1990-1999', 'moved in 1980-1989', 'moved in 1979 and earlier']
        seq.reverse()
    elif field_name == 'Age':
        seq = ['80 years and over', '70-79 years', '60-69 years', '50-59 years', '40-49 years', '31-39 years', 'under 30 years']
    elif field_name == 'HouseholdSize':
        seq = ['5+', '4', '3', '2', '1'].reverse()
    elif field_name == 'Income':
        seq = ['$200,000 and over', '$150,000-$199,999', '$100,000-$149,999', '$75,000-$99,000',
              '$50,000-$74,999', '$35,000-$49,999', 'Less than $35,000']
        seq.reverse()
    elif field_name == 'MarketValue':
        # repsonse too large for reasonable printing
            #df.rename(index = {"I don't live in a property owned by a family member" : 'Not owned by family'},
             #     inplace=True)
        seq = ['$2,000,000 and more', '$1,750,000-$1,999,999', '$1,500,000-$1,749,999',
              '$1,250,000-$1,499,999', '$1,000,000-$1,249,999', '$750,000-$999,999',
              '$500,000-$749,999', 'Less than $500,000']
        seq.reverse()
    else:
        raise 'Unknown field name %s' % field_name
    return seq


def plot_stress_roc(owndf, field_name, title_name):
    # remove observations which are na for the target field
    owndf = owndf[owndf[field_name].isna() == False]

    total_tested = owndf.StressBand.count()
    print 'total tested %d with target field %s filled in' % (total_tested, field_name)
    stressed_total = owndf[owndf.StressBand=='2. High Stress'].StressBand.count()
    print 'Of these, %d stressed' % stressed_total
    print owndf.StressBand.value_counts()
    print owndf[field_name].value_counts()
    seq = get_sequence(field_name)
    print 'sequence for', field_name, 'is', seq
    total_sum = np.zeros(len(seq)+1)
    total_recall = np.zeros(len(seq)+1)
    total_precision = np.ones(len(seq)+1)

    sum_group = 0
    sum_recall = 0
    #print 'debugging array buildup'
    for cntr, i in enumerate(seq):
        #print i
        sum_group += owndf[owndf[field_name]==i][field_name].count()
        #print sum_group
        total_sum[cntr+1] = sum_group
        sum_recall += owndf[(owndf[field_name]==i) & (owndf.StressBand=='2. High Stress')][field_name].count()
        total_recall[cntr+1] = sum_recall
        #print sum_recall
        total_precision[cntr+1] = 1.0 * sum_recall / sum_group

    print 'cum sums', total_sum
    print 'cum recall', total_recall

    #print 'total tested', total_tested
    false_alarms = total_sum - total_recall
    print 'false alarms', false_alarms
    fpr = false_alarms * 1.0 / (total_tested-stressed_total)
    print 'fpr', fpr

    print 'recall is total_recall divided by stressed total'
    print 'total_recall', total_recall
    #print 'stressed_total', stressed_total
    tpr = total_recall * 1.0 / stressed_total
    print 'recall', tpr

    print 'Random Guessing:'
    print '%d are stressed out of %d, total %% is %2.1f' % (stressed_total, total_tested, 100.0*stressed_total/total_tested)


    plot_roc('%s ROC Curve' % field_name, fpr, tpr)
    ax = plt.gca()

    for cntr, i in enumerate(seq[:-1]):
        y = tpr[cntr+1]
        x = fpr[cntr+1]
        ax.annotate(i,
                    xy=(x,y), xycoords='data',
                    xytext=(-32,0), textcoords='offset points',
                    arrowprops=dict(facecolor='black', shrink=0.05, width=1, headlength=6),
                    horizontalalignment='right', verticalalignment='bottom', fontsize=14)
    save_figure('%s_roc_curve.png' % field_name)
    
    
    # https://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html
    # In matplotlib < 1.5, plt.fill_between does not have a 'step' argument
    step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})
    
    
    recall = tpr
    precision = total_precision
        
    plt.rcParams.update({'font.size': 20})
#    plt.scatter(recall[1:], precision[1:], color='b', marker='o')
    plt.step(recall, precision, color='b', lw=1,
             where='pre', linestyle='--', marker='o', markersize=8)
    #plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title('%s Precision-Recall' % title_name)
    plt.grid(color='black', linestyle='-', linewidth=1, alpha=0.2)
    
    fig = matplotlib.pyplot.gcf()
    fig.set_size_inches(12,12)
    # annotations
    ax = plt.gca()
    ax.xaxis.set_major_locator(mticker.MaxNLocator(11))
    ax.yaxis.set_major_locator(mticker.MaxNLocator(11))
    vals = ax.get_yticks()
    ax.set_yticklabels(['{:,.0%}'.format(x) for x in vals])
    vals = ax.get_xticks()
    ax.set_xticklabels(['{:,.0%}'.format(x) for x in vals])
    
    
    for cntr, i in enumerate(seq):
        y = precision[cntr+1]
        x = recall[cntr+1]
        posn_base = len(i)
        ax.annotate(i,
                    xy=(x,y), xycoords='data',
                    xytext=(60,12*(len(seq)-cntr)), textcoords='offset points',
                    arrowprops=dict(facecolor='black', shrink=0.05, width=1, headlength=6),
                    horizontalalignment='right', verticalalignment='bottom', fontsize=10)
    save_figure('%s_precision_recall.png' % field_name)

    
    


# - --------- use owndf only from here on ---------

owndf = plotdf[(plotdf['OwnRent']=='Own') & (owndf.StressBand.isna() == False)]

plot_stress_roc(owndf, 'MarketValue', 'Home Value')
plot_stress_roc(owndf, 'Tenure', 'Tenure in Lexington')
plot_stress_roc(owndf, 'Age', 'Age')
plot_stress_roc(owndf, 'Income', 'Income')